In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='misc')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.50 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/misc/65f657e72546423088162d387521a8bf



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint
from Model import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='debug', type=str, help="name of the log")
parser.add_argument("-gpu", default='0', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=2, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        arprint=ArtPrint(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 2
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/debug
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images already exists, skipping download
[0.10406644 0.89593356]
Instructions for updating:
Colocations handled automatically by placer.
GGG
(?, 32, 128, 1)
(?, 32, 128, 1)
(?, 32, 128, 64)
111111
(?, 16, 64, 64)
(?, 16, 64, 64)
22222
(?, 8, 32, 64)
(?, 8, 32, 64)
33333
(?, 4, 16, 64)
(?, 4, 16, 64)
44444
(?, 2, 8, 64)
(?, 2, 8, 64)
d4444444
(?, 4, 16, 64)
d333333
(?, 8, 32, 64)
d22222
(?, 16, 64, 64)
d111111
(?, 32, 128, 64)
cv
(?, 32, 128, 2)
(2,)
(?, 32, 128, 2)
loss: ()
INFO:tensorflow:Summary name conv_classifier/weight_loss (raw) is illegal; using conv_classifier/weight_loss__raw_ instead.
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead

COMET ERROR: Failed to extract parameters from Estimator.init()


toto_loss_shape: Tensor("loss/total_loss:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Python: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
Tensorflow: 1.12.0-rc0
Ran global_variables_initializer
Epoch: 1  Training...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


TRAIN: Batch: 0.0 Loss: 0.19717042
accuracy = 0.410015
mean IU  = 0.241899
    class # 0 accuracy = 0.362084 
    class # 1 accuracy = 0.778532 
TRAIN: Batch: 0.003908998514580564 Loss: 0.11671237
accuracy = 0.786972
mean IU  = 0.520551
    class # 0 accuracy = 0.790620 
    class # 1 accuracy = 0.756069 
TRAIN: Batch: 0.007817997029161129 Loss: 0.10590297
accuracy = 0.801319
mean IU  = 0.570642
    class # 0 accuracy = 0.818177 
    class # 1 accuracy = 0.713200 
TRAIN: Batch: 0.011726995543741693 Loss: 0.077968456
accuracy = 0.773207
mean IU  = 0.506031
    class # 0 accuracy = 0.774452 
    class # 1 accuracy = 0.762432 
TRAIN: Batch: 0.015635994058322257 Loss: 0.1120324
accuracy = 0.366681
mean IU  = 0.209582
    class # 0 accuracy = 0.340414 
    class # 1 accuracy = 0.539644 
TRAIN: Batch: 0.019544992572902823 Loss: 0.07257057
accuracy = 0.328100
mean IU  = 0.182690
    class # 0 accuracy = 0.304426 
    class # 1 accuracy = 0.524642 
TRAIN: Batch: 0.023453991087483386 Loss: 0.06

...

TRAIN: Batch: 0.5980767727308264 Loss: 0.025231063
accuracy = 0.946216
mean IU  = 0.785092
    class # 0 accuracy = 0.945382 
    class # 1 accuracy = 0.954093 
TRAIN: Batch: 0.601985771245407 Loss: 0.034701675
accuracy = 0.912396
mean IU  = 0.722353
    class # 0 accuracy = 0.909203 
    class # 1 accuracy = 0.938037 
TRAIN: Batch: 0.6058947697599875 Loss: 0.026079942
accuracy = 0.937007
mean IU  = 0.757646
    class # 0 accuracy = 0.934188 
    class # 1 accuracy = 0.964894 
TRAIN: Batch: 0.6098037682745681 Loss: 0.03765894
accuracy = 0.930262
mean IU  = 0.747488
    class # 0 accuracy = 0.929329 
    class # 1 accuracy = 0.938730 
TRAIN: Batch: 0.6137127667891487 Loss: 0.034196477
accuracy = 0.938646
mean IU  = 0.758570
    class # 0 accuracy = 0.939660 
    class # 1 accuracy = 0.928740 
TRAIN: Batch: 0.6176217653037291 Loss: 0.019853951
accuracy = 0.959886
mean IU  = 0.834327
    class # 0 accuracy = 0.959134 
    class # 1 accuracy = 0.966414 
TRAIN: Batch: 0.6215307638183097 Los

TRAIN: Batch: 0.7974356969744352 Loss: 0.025471603
accuracy = 0.938217
mean IU  = 0.782315
    class # 0 accuracy = 0.935295 
    class # 1 accuracy = 0.961572 
TRAIN: Batch: 0.8013446954890158 Loss: 0.034938153
accuracy = 0.958349
mean IU  = 0.822370
    class # 0 accuracy = 0.956343 
    class # 1 accuracy = 0.977419 
TRAIN: Batch: 0.8052536940035963 Loss: 0.026155688
accuracy = 0.940417
mean IU  = 0.785001
    class # 0 accuracy = 0.936898 
    class # 1 accuracy = 0.969594 
TRAIN: Batch: 0.8091626925181769 Loss: 0.023668407
accuracy = 0.943256
mean IU  = 0.792208
    class # 0 accuracy = 0.938128 
    class # 1 accuracy = 0.986611 
TRAIN: Batch: 0.8130716910327574 Loss: 0.04295531
accuracy = 0.902232
mean IU  = 0.725293
    class # 0 accuracy = 0.894099 
    class # 1 accuracy = 0.955973 
TRAIN: Batch: 0.8169806895473379 Loss: 0.040342487
accuracy = 0.916815
mean IU  = 0.725811
    class # 0 accuracy = 0.918834 
    class # 1 accuracy = 0.900510 
TRAIN: Batch: 0.8208896880619185 Lo

TRAIN: Batch: 0.996794621218044 Loss: 0.028899906
accuracy = 0.950585
mean IU  = 0.803613
    class # 0 accuracy = 0.948908 
    class # 1 accuracy = 0.965643 
Validating NN


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 95.107282%. Class 0 capture: 95.216644%. Class 1 capture: 94.162689%
Character error rate improved, save model
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Done with training at epoch 2 sigoptObservation=0.9510728236211811
